# Two-table Verbs

This is a copy of the dplyr vignette [Two Table Verbs](https://cran.r-project.org/web/packages/dplyr/vignettes/two-table.html) showing how to perform the same operations using pplyr/pandas.

Primarily, these verbs deal with joining two tables together.  Pandas includes a very nice ```merge``` function that is almost identical to the same function in R.  This function takes the form:

```
pd.merge(left, right, how='inner', 
           on=None, left_on=None, right_on=None, 
           left_index=False, right_index=False, 
           sort=False, suffixes=('_x', '_y'), 
           copy=True, indicator=False, validate=None)
```

The pplyr library introduces the following verbs that basically just wrap this function and provide different values for the 'how' parameter:

* inner_join
* left_join
* right_join
* outer_join

R's dplyr library includes three famlies of verbs that work with two tables at a time:

* Mutating joins, which add new variables to one table from matching rows in another.
* Filtering joins, which filter observations from one table based on whether or not they match an observation in the other table.
* Set operations, which combine the observations in the data sets as if they were set elements.

So far pplyr only implements mutating joins.  We have not yet done filtering joins or set operations.

## Imports

In [9]:
import sys
if ".." not in sys.path:
    sys.path.append("..")

import pplyr

In [19]:
import pandas as pd
from nycflights13 import flights, airlines, weather, planes, airports

## Mutating Joins
Mutating joins allow you to combine variables from multiple tables. For example, take the nycflights13 data. In one table we have flight information with an abbreviation for carrier, and in another we have a mapping between abbreviations and full names. You can use a join to add the carrier names to the flight data:

In [13]:
flights2 = flights.pipe(pplyr.select, 
                        ["year","month","day", "hour", 
                         "origin", "dest", "tailnum", "carrier"])

pplyr.left_join(flights2, airlines).head()

,year,month,day,hour,origin,dest,tailnum,carrier,name
0,2013,1,1,5,EWR,IAH,N14228,UA,United Air Lines Inc.
1,2013,1,1,5,LGA,IAH,N24211,UA,United Air Lines Inc.
2,2013,1,1,5,JFK,MIA,N619AA,AA,American Airlines Inc.
3,2013,1,1,5,JFK,BQN,N804JB,B6,JetBlue Airways
4,2013,1,1,6,LGA,ATL,N668DN,DL,Delta Air Lines Inc.


### Controlling how the tables are matched

As well as x and y, each mutating join takes an argument by that controls which variables are used to match observations in the two tables. There are a few ways to specify it, as I illustrate below with various tables from nycflights13:

* NULL, the default. dplyr will will use all variables that appear in both tables, a __natural__ join. For example, the flights and weather tables match on their common variables: year, month, day, hour and origin.

In [14]:
pplyr.left_join(flights2, weather).head()

,year,month,day,hour,origin,dest,tailnum,carrier,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
0,2013,1,1,5,EWR,IAH,N14228,UA,39.02,28.04,64.43,260.0,12.65858,NaN,0.0,1011.9,10.0,2013-01-01T10:00:00Z
1,2013,1,1,5,LGA,IAH,N24211,UA,39.92,24.98,54.81,250.0,14.96014,21.86482,0.0,1011.4,10.0,2013-01-01T10:00:00Z
2,2013,1,1,5,JFK,MIA,N619AA,AA,39.02,26.96,61.63,260.0,14.96014,NaN,0.0,1012.1,10.0,2013-01-01T10:00:00Z
3,2013,1,1,5,JFK,BQN,N804JB,B6,39.02,26.96,61.63,260.0,14.96014,NaN,0.0,1012.1,10.0,2013-01-01T10:00:00Z
4,2013,1,1,6,LGA,ATL,N668DN,DL,39.92,24.98,54.81,260.0,16.11092,23.01560,0.0,1011.7,10.0,2013-01-01T11:00:00Z


* A character vector, ```on = "x"```. Like a natural join, but uses only some of the common variables. For example, ```flights``` and ```planes``` have ```year``` columns, but they mean different things so we only want to join by ```tailnum```.

Note that the year columns in the output are disambiguated with a suffix.

In [15]:
pplyr.left_join(flights2, planes, on = "tailnum").head()

,year_x,month,day,hour,origin,dest,tailnum,carrier,year_y,type,manufacturer,model,engines,seats,speed,engine
0,2013,1,1,5,EWR,IAH,N14228,UA,1999.0,Fixed wing multi engine,BOEING,737-824,2.0,149.0,NaN,Turbo-fan
1,2013,1,1,5,LGA,IAH,N24211,UA,1998.0,Fixed wing multi engine,BOEING,737-824,2.0,149.0,NaN,Turbo-fan
2,2013,1,1,5,JFK,MIA,N619AA,AA,1990.0,Fixed wing multi engine,BOEING,757-223,2.0,178.0,NaN,Turbo-fan
3,2013,1,1,5,JFK,BQN,N804JB,B6,2012.0,Fixed wing multi engine,AIRBUS,A320-232,2.0,200.0,NaN,Turbo-fan
4,2013,1,1,6,LGA,ATL,N668DN,DL,1991.0,Fixed wing multi engine,BOEING,757-232,2.0,178.0,NaN,Turbo-fan


* Separate vectors for the left and right tables, specified by ```left_on``` and ```right_on```.

Each flight has an origin and destination airport, so we need to specify which one we want to join to:

In [16]:
pplyr.left_join(flights2, airports, left_on="dest", right_on="faa").head()

,year,month,day,hour,origin,dest,tailnum,carrier,faa,name,lat,lon,alt,tz,dst,tzone
0,2013,1,1,5,EWR,IAH,N14228,UA,IAH,George Bush Intercontinental,29.984433,-95.341442,97.0,-6.0,A,America/Chicago
1,2013,1,1,5,LGA,IAH,N24211,UA,IAH,George Bush Intercontinental,29.984433,-95.341442,97.0,-6.0,A,America/Chicago
2,2013,1,1,5,JFK,MIA,N619AA,AA,MIA,Miami Intl,25.793250,-80.290556,8.0,-5.0,A,America/New_York
3,2013,1,1,5,JFK,BQN,N804JB,B6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013,1,1,6,LGA,ATL,N668DN,DL,ATL,Hartsfield Jackson Atlanta Intl,33.636719,-84.428067,1026.0,-5.0,A,America/New_York


In [17]:
pplyr.left_join(flights2, airports, left_on="origin", right_on="faa").head()

,year,month,day,hour,origin,dest,tailnum,carrier,faa,name,lat,lon,alt,tz,dst,tzone
0,2013,1,1,5,EWR,IAH,N14228,UA,EWR,Newark Liberty Intl,40.692500,-74.168667,18,-5,A,America/New_York
1,2013,1,1,5,LGA,IAH,N24211,UA,LGA,La Guardia,40.777245,-73.872608,22,-5,A,America/New_York
2,2013,1,1,5,JFK,MIA,N619AA,AA,JFK,John F Kennedy Intl,40.639751,-73.778925,13,-5,A,America/New_York
3,2013,1,1,5,JFK,BQN,N804JB,B6,JFK,John F Kennedy Intl,40.639751,-73.778925,13,-5,A,America/New_York
4,2013,1,1,6,LGA,ATL,N668DN,DL,LGA,La Guardia,40.777245,-73.872608,22,-5,A,America/New_York


### Types of Joins

There are four types of mutating join, which differ in their behaviour when a match is not found. We’ll illustrate each with a simple example:

In [21]:
df1 = pd.DataFrame({"x": [1, 2], "y": [2, 1]})
df2 = pd.DataFrame({"x": [3, 1], "a": 10, "b": "a"})

* ```inner_join(x, y)``` only includes observations that match in both x and y.

In [22]:
df1.pipe(pplyr.inner_join, df2)

,x,y,a,b
0,1,2,10,a


* ```left_join(x, y)``` includes all observations in x, regardless of whether they match or not. This is the most commonly used join because it ensures that you don’t lose observations from your primary table.

In [23]:
df1.pipe(pplyr.left_join, df2)

,x,y,a,b
0,1,2,10.0,a
1,2,1,NaN,NaN


* ```right_join(x, y)``` includes all observations in y. It’s equivalent to ```left_join(y, x)```, but the columns and rows will be ordered differently.

In [24]:
df1.pipe(pplyr.right_join, df2)

,x,y,a,b
0,3,NaN,10,a
1,1,2.0,10,a


In [25]:
df2.pipe(pplyr.left_join, df1)

,x,a,b,y
0,3,10,a,NaN
1,1,10,a,2.0


* ```outer_join()``` includes all observations from x and y.

In [28]:
df1.pipe(pplyr.outer_join, df2)

,x,y,a,b
0,1,2.0,10.0,a
1,2,1.0,NaN,NaN
2,3,NaN,10.0,a


The left, right and full joins are collectively know as outer joins. When a row doesn’t match in an outer join, the new variables are filled in with missing values.

### Observations

While mutating joins are primarily used to add new variables, they can also generate new observations. If a match is not unique, a join will add all possible combinations (the Cartesian product) of the matching observations:

In [29]:
df1 = pd.DataFrame({"x": [1,1,2], "y": [1,2,3]})
df2 = pd.DataFrame({"x": [1,1,2], "z": ["a","b","a"]})

df1.pipe(pplyr.left_join, df2)

,x,y,z
0,1,1,a
1,1,1,b
2,1,2,a
3,1,2,b
4,2,3,a


## Filtering Joins

These are not yet included in the library

TODO?  

## Set Operations

These are not yet included in the library

TODO?  

## Multiple-table Verbs

These are not yet included in the library

TODO?  